In [386]:
# Fall 2012 6.034 Lab 2: Search
#
# Your answers for the true and false questions will be in the following form.  
# Your answers will look like one of the two below:
#ANSWER1 = True
#ANSWER1 = False

# 1: True or false - Hill Climbing search is guaranteed to find a solution
#    if there is a solution
ANSWER1 = False

# 2: True or false - Best-first search will give an optimal search result
#    (shortest path length).
#    (If you don't know what we mean by best-first search, refer to
#     http://courses.csail.mit.edu/6.034f/ai3/ch4.pdf (page 13 of the pdf).)
ANSWER2 = False

# 3: True or false - Best-first search and hill climbing make use of
#    heuristic values of nodes.
ANSWER3 = True

# 4: True or false - A* uses an extended-nodes set.
ANSWER4 = True

# 5: True or false - Breadth first search is guaranteed to return a path
#    with the shortest number of nodes.
ANSWER5 = True

# 6: True or false - The regular branch and bound uses heuristic values
#    to speed up the search for an optimal path.
ANSWER6 = False

# Import the Graph data structure from 'search.py'
# Refer to search.py for documentation
from search import Graph
from graphs import GRAPH1
from graphs import NEWGRAPH1
from graphs import NEWGRAPH2

In [339]:
## Optional Warm-up: BFS and DFS
# If you implement these, the offline tester will test them.
# If you don't, it won't.
# The online tester will not test them.

def bfs(graph, start, goal):
    #create a queue with starting root node in it
    queue=[[start]]

    #while there is path to discover keep searchig
    while len(queue)!=0:
        path=[]
        #get out the first path to explore
        first_path=queue.pop(0)
        
        #if any of the path finishes in the goal node break out and return the path
        if first_path[-1]==goal:
            break
        

        #explore all nodes in the first path and append it to the tail of the path
        for node in graph.get_connected_nodes(first_path[-1]):
            if node in first_path:
                continue
            path.append(first_path+[node])
        #add the path to the tail of the queue
        queue=queue+path
    edges=[]
    for i in range(len(first_path)):
        if i!=len(first_path)-1:
            edges.append(graph.get_edge(first_path[i], first_path[i+1]))
            
    #if the path did not end in goal node then return empty list
    if first_path[-1]!=goal:
        return []
    return first_path

In [340]:
bfs(GRAPH1, 'Mirrored Room','Common Area')

['Mirrored Room', 'Stairs', 'Common Area']

In [413]:
## Once you have completed the breadth-first search,
## this part should be very simple to complete.
def dfs(graph, start, goal):
    #create a queue with starting root node in it
    queue=[[start]]
    
    #while there is path to discover keep searchig
    while len(queue)!=0:
        path=[]
        #get out the first path to explore
        first_path=queue.pop(0)
        
        #if any of the path finishes in the goal node break out and return the path
        if first_path[-1]==goal:
            break

        #explore all nodes in the first path and append it to the tail of the path
        for node in graph.get_connected_nodes(first_path[-1]):
            #if the node is already in the first path look at the other node
            if  node in first_path:
                continue
            #in contrast to bfs append the first path to the front of the queue
            path=[first_path+[node]]+path
        
        #add the explored paths into the front of the queue
        queue=path+queue

    #if the path did not end in goal node then return empty list
    if first_path[-1]!=goal:
        return []
    return first_path

In [414]:
dfs(GRAPH1, 'Mirrored Room','Common Area')

['Mirrored Room', 'Statues', 'Stairs', 'Common Area']

In [411]:
## Now we're going to add some heuristics into the search.  
## Remember that hill-climbing is a modified version of depth-first search.
## Search direction should be towards lower heuristic values to the goal.

def hill_climbing(graph, start, goal):
    #create a path with starting root node in it
    queue=[[graph.get_heuristic(start, goal),[start]]]
    #while there is path to discover keep searchig
    while len(queue)!=0:
        
        path=[]
        
        #get out the first path to explore
        first_path=queue.pop(0)
        #remove the heurisctic distance to update
        first_path=first_path[-1]
        #explore all nodes in the first path and append it to the tail of the queue path
        
        #if any of the path finishes in the goal node break out and return the path
        if first_path[-1]==goal:
            break
       
        for node in graph.get_connected_nodes(first_path[-1]):
            #if the node is already in the first path or the node is the starting node look at the other node
            if node in first_path:
                continue
            
            node_heuristic=graph.get_heuristic(node, goal)
            #in contrast to bfs or dfs we need to append the first path by sorting based on heursitics
            path=[[node_heuristic,first_path+[node]]]+path
            
        #sort the path based on heuristics distance inside the nested list.
        path=sorted(path)
        #add the path to the front of the queue
        queue=path+queue

    #if the path did not end in goal node then return empty list
    if first_path[-1]!=goal:
        return []
    return first_path

In [412]:
hill_climbing(NEWGRAPH1,'H','G')

['H', 'D', 'B', 'A', 'S', 'C', 'E', 'G']

In [348]:
hill_climbing(NEWGRAPH1,'F','G')
# ['F', 'B', 'D', 'C', 'E', 'G']

['F', 'B', 'D', 'C', 'E', 'G']

In [349]:
hill_climbing(GRAPH1, 'Mirrored Room','Common Area')

['Mirrored Room', 'Stairs', 'Common Area']

In [408]:
## Now we're going to implement beam search, a variation on BFS
## that caps the amount of memory used to store paths.  Remember,
## we maintain only k candidate paths of length n in our agenda at any time.
## The k top candidates are to be determined using the 
## graph get_heuristic function, with lower values being better values.
def beam_search(graph, start, goal, beam_width):
    #create a path with starting root node in it
    queue=[[graph.get_heuristic(start, goal),[start]]]
    #while there is path to discover keep searchig
    while len(queue)!=0:
        
        path=[]
        
        #get out the first path to explore
        first_path=queue[0]
        #remove the heurisctic distance to update
        first_path=first_path[-1]
        #if any of the path finishes in the goal node break out and return the path
        if first_path[-1]==goal:
            break
        for branch in queue:
            #branch is the list including heuristic value and the pathway. [heuristic distance,[pathway]]
            #so by telling branch[-1][-1] we are extracting the pathway's final node.
            for node in graph.get_connected_nodes(branch[-1][-1]):
                #if the node is already in the branch pathway then pass to the next iteration (node)
                if node in branch[-1]:
                    continue
                #get the heuristic distance of the new node
                node_heuristic=graph.get_heuristic(node, goal)
                #create a new pathway with new heuristic value
                path=[[node_heuristic,branch[-1]+[node]]]+path
            
        #sort the path based on heuristics distance inside the nested list.
        path=sorted(path)
        #keep beam widths (k) number of elements of the path at the level n. assign it to the queue
        queue=path[:beam_width]
    
    #if the path did not end in goal node then return empty list
    if first_path[-1]!=goal:
        return []
    return first_path

In [409]:
beam_search(GRAPH1, 'Mirrored Room','Common Area',4)

['Mirrored Room', 'Stairs', 'Common Area']

In [410]:
beam_search(NEWGRAPH1,'F','G',2)

['F', 'B', 'A', 'S', 'C', 'E', 'G']

In [421]:
## Now we're going to try optimal search.  The previous searches haven't
## used edge distances in the calculation.

## This function takes in a graph and a list of node names, and returns
## the sum of edge lengths along the path -- the total distance in the path.
def path_length(graph, node_names):
    if len(node_names)==1:
        return 0
    length=0
    for i in range(len(node_names)):
        if i!=len(node_names)-1:
            edge=graph.get_edge(node_names[i], node_names[i+1])
            length+=edge.length
    return length

In [423]:
path_length(NEWGRAPH1,['F'])

0

In [469]:
def branch_and_bound(graph, start, goal):
     #create a queue with starting root node in it
    queue=[[0,[start]]]
    
    #while there is path to discover keep searchig
    while len(queue)!=0:
        path=[]
        #get out the first path to explore
        first_path=queue.pop(0)
        #the paths in the queue in the format of [[pathlengt,[path]]]
        #so by making first_path[-1] we are extracting only the path list
        first_path=first_path[-1]
        #if any of the path finishes in the goal node break out and return the path
        if first_path[-1]==goal:
            break

        #explore all nodes in the first path and append it to the tail of the path
        for node in graph.get_connected_nodes(first_path[-1]):
            #if the node is already in the first path look at the other node
            if  node in first_path:
                continue
            #in contrast to bfs append the first path to the front of the queue
            path=[first_path+[node]]+path
        #for the paths gathered find the new length and update the queue with appropriate lengths
        for p in path:
            length=path_length(graph,p)
            queue.append([length,p])
            
        #sort the queue based on the path length
        queue=sorted(queue)


    #if the path did not end in goal node then return empty list
    if first_path[-1]!=goal:
        return []
    return first_path

In [470]:
branch_and_bound(NEWGRAPH1,'F','G')

['F', 'B', 'D', 'E', 'G']

In [503]:
def a_star(graph, start, goal):
     #create a queue with starting root node in it
    queue=[[0,[start]]]
    
    #create a set of queue that will store all traversed nodes. 
    #it also will prevent to extend already visited nodes.
    set_queue=set(start)
    
    #while there is path to discover keep searchig
    while len(queue)!=0:
        #get out the first path to explore
        first_path=queue.pop(0)
        #the paths in the queue in the format of [[pathlengt,[path]]]
        #so by making first_path[-1] we are extracting only the path list
        first_path=first_path[-1]
        #if any of the path finishes in the goal node break out and return the path
        if first_path[-1]==goal:
            break

        #explore all nodes in the first path and append it to the tail of the path
        for node in graph.get_connected_nodes(first_path[-1]):
            #if the node is already in the first path look at the other node
            if  node in first_path or node in set_queue:
                continue
            #in contrast to bfs append the first path to the front of the queue
            path=first_path+[node]
            #calculate the length of the current path
            length=path_length(graph,path)
            #get the heuristic distance of the current node
            node_heuristic=graph.get_heuristic(node, goal)
            #calculate the estimated total distance
            estimate=length+node_heuristic
            #add the estimated distance and current path to the queue
            queue.append([estimate,path])    
            #add the node to the set
            set_queue.add(node)
            
        #sort the queue based on the path length
        queue=sorted(queue)
        
    #if the path did not end in goal node then return empty list
    if first_path[-1]!=goal:
        return []
    return first_path

In [504]:
a_star(NEWGRAPH1,'F','G')

['F', 'B', 'D', 'E', 'G']

In [534]:
## It's useful to determine if a graph has a consistent and admissible
## heuristic.  You've seen graphs with heuristics that are
## admissible, but not consistent.  Have you seen any graphs that are
## consistent, but not admissible?

def is_admissible(graph, goal):
    
    for node in graph.nodes:
        shortest_path=a_star(graph,node,goal)
        shortest_length=path_length(graph,shortest_path)
        heuristic_distance=graph.get_heuristic(node, goal)
        if heuristic_distance>shortest_length:
            return False
    return True

In [535]:
is_admissible(NEWGRAPH1,'A')

True

In [536]:
def is_consistent(graph, goal):
    
    for node1 in graph.nodes:
        for node2 in graph.get_connected_nodes(node1):
            edge=graph.get_edge(node1, node2)
            edge_length=edge.length
            h1=graph.get_heuristic(node1, goal)
            h2=graph.get_heuristic(node2, goal)
            
            if edge_length<abs(h1-h2):
                return False
    return True

In [537]:
is_consistent(NEWGRAPH1, 'G')

True